In [1]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\thoma\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
import os
import re
import json
import requests
import nltk
from typing import List, Dict, Any
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

In [3]:
# Εισαγωγή απαραίτητων βιβλιοθηκών (Importing necessary libraries)
# os: Λειτουργίες συστήματος και διαδρομών (System and path operations)
# re: Επεξεργασία κειμένου με κανονικές εκφράσεις (Text processing with regular expressions)
# json: Επεξεργασία δεδομένων JSON (JSON data handling)
# requests: Πραγματοποίηση HTTP αιτημάτων (Making HTTP requests)
# typing: Προσθήκη τύπων δεδομένων (Adding type hints)
# BeautifulSoup: Ανάλυση HTML και XML (HTML and XML parsing)
# stopwords: Λίστα κοινών λέξεων που αγνοούνται (List of common words to ignore)
# word_tokenize: Διαχωρισμός κειμένου σε λέξεις (Splitting text into words)
# WordNetLemmatizer: Αναγωγή λέξεων στη βασική τους μορφή (Reducing words to their base form)

In [4]:
#Κλάση για την εξαγωγή και επεξεργασία περιεχομένου από τη Wikipedia.
class WikipediaCrawler:
    def __init__(self):
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        
    def get_wikipedia_content(self, subject: str) -> Dict[str, Any]:
        # Μορφοποίηση του θέματος για χρήση στο URL
        formatted_subject = subject.strip().replace(" ", "_")
        url = f'https://en.wikipedia.org/wiki/{formatted_subject}'
        
        try:
            # Λήψη και επεξεργασία της σελίδας
            page = requests.get(url)
            page.raise_for_status()
            soup = BeautifulSoup(page.text, 'html.parser')
            
            #Επιστρέφει:Dict[str, Any]: Λεξικό με τον τίτλο και τις παραγράφους της σελίδας
            return {
                'title': soup.title.string,
                'paragraphs': [p.get_text() for p in soup.find_all('p')]
            }
        except requests.RequestException as e:
            print(f"Σφάλμα κατά τη λήψη της σελίδας Wikipedia: {e}")
            return None

    def preprocess_text(self, text: str) -> List[str]:
        """
        Επεξεργάζεται το κείμενο αφαιρώντας ειδικούς χαρακτήρες, μετατρέποντας
        σε πεζά, αφαιρώντας stop words και εφαρμόζοντας lemmatization.
        """
        # Αφαίρεση ειδικών χαρακτήρων και περιττών κενών
        text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
        text = re.sub(r'\s+', ' ', text).strip()
        
        # Διαχωρισμός σε tokens και επεξεργασία
        tokens = word_tokenize(text)
        return [
            self.lemmatizer.lemmatize(word.lower(),'v')
            for word in tokens
            if word.lower() not in self.stop_words
        ]

    def process_wikipedia_data(self, data: Dict[str, Any]) -> Dict[str, Any]:
        """
        Επεξεργάζεται τα δεδομένα της Wikipedia εφαρμόζοντας προεπεξεργασία
        σε όλες τις παραγράφους.
        """
        return {
            'title': data['title'],
            'paragraphs': [
                self.preprocess_text(paragraph)
                for paragraph in data['paragraphs']
            ]
        }

In [5]:
# Κλάση για εξαγωγή περιεχομένου από Wikipedia
# Αρχικοποίηση εργαλείων επεξεργασίας κειμένου
# Προετοιμασία λημματοποιητή και λέξεων στάσης
# Μέθοδος για λήψη περιεχομένου Wikipedia
# Μετατροπή θέματος σε μορφή URL
# Αποστολή αιτήματος στον διακομιστή
# Χειρισμός πιθανών σφαλμάτων
# Εξαγωγή τίτλου και παραγράφων
# Επιστροφή λεξικού με πληροφορίες σελίδας

In [6]:
# Επεξεργασία κειμένου με πολλαπλά βήματα
# Αφαίρεση ειδικών χαρακτήρων με κανονική έκφραση
# Καθαρισμός πλεοναζόντων κενών χαρακτήρων
# Διαχωρισμός κειμένου σε λέξεις
# Μετατροπή σε πεζά και εφαρμογή λημματοποίησης
# Αφαίρεση λέξεων στάσης
# Επιστροφή επεξεργασμένων λέξεων

In [7]:
# Επεξεργασία δεδομένων Wikipedia
# Διατήρηση του αρχικού τίτλου
# Προεπεξεργασία κάθε παραγράφου με τη μέθοδο preprocess_text
# Επιστροφή λεξικού με τον τίτλο και τις επεξεργασμένες παραγράφους

In [8]:
class DataStorage:
    """
    Διαχειρίζεται την αποθήκευση και φόρτωση των δεδομένων.
    """
    @staticmethod
    def save_to_json(data: Dict[str, Any], filename: str) -> None:
        """
        Αποθηκεύει δεδομένα σε αρχείο JSON, προσθέτοντάς τα αν το αρχείο υπάρχει.
        """
        try:
            # Φόρτωση υπαρχόντων δεδομένων ή δημιουργία κενής λίστας
            existing_data = []
            if os.path.exists(filename):
                with open(filename, 'r', encoding='utf-8') as f:
                    try:
                        existing_data = json.load(f)
                    except json.JSONDecodeError:
                        pass

            # Προσθήκη νέων δεδομένων και αποθήκευση
            existing_data.append(data)
            with open(filename, 'w', encoding='utf-8') as f:
                json.dump(existing_data, f, indent=4, ensure_ascii=False)
        except Exception as e:
            print(f"Σφάλμα κατά την αποθήκευση στο {filename}: {e}")

In [9]:
# Διαχείριση της αποθήκευσης και φόρτωσης δεδομένων
# Στατική μέθοδος για αποθήκευση δεδομένων σε αρχείο JSON
# Έλεγχος αν το αρχείο υπάρχει και φόρτωση των υπαρχόντων δεδομένων
# Αν το αρχείο δεν είναι έγκυρο JSON, παρακάμπτουμε το σφάλμα
# Προσθήκη των νέων δεδομένων στο τέλος της λίστας
# Αποθήκευση των δεδομένων πίσω στο αρχείο JSON με μορφοποίηση
# Αν παρουσιαστεί κάποιο σφάλμα κατά την αποθήκευση, εκτύπωση του σφάλματος

In [10]:
class InvertedIndex:
    def __init__(self):
        """
        Αρχικοποίηση του ανεστραμμένου ευρετηρίου.
        Δημιουργούμε ένα dictionary που θα αποθηκεύει για κάθε λέξη
        τα έγγραφα στα οποία εμφανίζεται και τη συχνότητά της.
        """
        self.index = {}  # το κυρίο ευρετηρίο
        self.documents = {}  # Αποθυκευσή εγγραφώ με τα IDs τους

    def load_processed_data(self, filename: str) -> None:
        try:
            with open(filename, 'r', encoding='utf-8') as f:
                self.documents = json.load(f)
        except Exception as e:
            print(f"Σφάλμα κατα την φορτώση αρχείου {filename} : {e}")

    def build_index(self) -> None:
            """
            Δημιουργία του ανεστραμμένου ευρετηρίου από τα επεξεργασμένα δεδομένα.
            Για κάθε λέξη, αποθηκεύουμε:
            - Σε ποια έγγραφα εμφανίζεται
            - Πόσες φορές εμφανίζεται σε κάθε έγγραφο
            """
            try:
                # Για κάθε έγγραφο στα δεδομένα μας
                for doc_id, doc_data in enumerate(self.documents):
                    # Παίρνουμε τον τίτλο και τις επεξεργασμένες παραγράφους
                    title = doc_data['title']
                    paragraphs = doc_data['paragraphs']

                    # Για κάθε παράγραφο στο έγγραφο
                    for paragraph in paragraphs:
                        for word in paragraph:
                            # Αν η λέξη δεν υπάρχει στο ευρετήριο, την προσθέτουμε
                            if word not in self.index:
                                self.index[word] = {
                                    'documents': {},  # Έγγραφα που περιέχουν τη λέξη
                                    'total_occurrences': 0  # Συνολικές εμφανίσεις
                                }

                            # Ενημερώνουμε τη συχνότητα για το συγκεκριμένο έγγραφο
                            if doc_id not in self.index[word]['documents']:
                                self.index[word]['documents'][doc_id] = {
                                    'count': 1,
                                    'title': title
                                }
                            else:
                                self.index[word]['documents'][doc_id]['count'] += 1

                            # Αυξάνουμε το συνολικό μετρητή εμφανίσεων
                            self.index[word]['total_occurrences'] += 1
            except Exception as e:
                print(f"Σφάλμα κατά τη δημιουργία του ευρετηρίου: {e}")


    def get_format_for_term(self, term: str) -> Dict[str, Any]:
            """
            Επιστρέφει τα έγγραφα που περιέχουν τον συγκεκριμένο όρο.
            """
            return self.index.get(term, {})

In [11]:
# Αρχικοποίηση του ανεστραμμένου ευρετηρίου
# Δημιουργία του ευρετηρίου και αποθήκευση των εγγράφων με τα IDs τους

# Φόρτωση των επεξεργασμένων δεδομένων από αρχείο JSON
# Αν υπάρξει σφάλμα κατά την φόρτωση του αρχείου, εκτύπωση του σφάλματος

# Δημιουργία του ανεστραμμένου ευρετηρίου από τα δεδομένα
# Για κάθε έγγραφο, αποθηκεύουμε τις λέξεις και τη συχνότητά τους
# Ενημέρωση του ευρετηρίου με τα έγγραφα και τις εμφανίσεις κάθε λέξης

# Επιστροφή των εγγράφων που περιέχουν τον συγκεκριμένο όρο

In [12]:
def main():
    crawler = WikipediaCrawler()
    storage = DataStorage()

    """
    Εαν τα αρχεία υπάρχουν από προηγούμενες αναζητήσεις διαγράφονται.
    """
    if os.path.exists('webpage_data.json'):
        os.remove('webpage_data.json')
    if os.path.exists('processed_webpage_data.json'):
        os.remove('processed_webpage_data.json')
    
    subjects = [
        "Python_programming_language",
        "Artificial_intelligence",
        "Machine_learning",
        "Data_science",
        "Computer_programming",
        "Java_programming_language",
        "Database_management_system",
        "Cloud_computing",
        "Cybersecurity",
        "Web_development",
        "Computer_networks",
        "Software_engineering",
        "Internet_of_things",
        "Big_data",
        "Computer_graphics",
        "Operating_system",
        "Mobile_computing",
        "Computer_architecture",
        "Information_security",
        "Distributed_computing",
        "Algorithm",
        "Data_mining",
        "Blockchain",
        "Deep_learning"
        ]
    for subject in subjects:
        print(f"Συλλογή Δεδομλενων για :{subject}")
        raw_data = crawler.get_wikipedia_content(subject)
        if raw_data:
            storage.save_to_json(raw_data, 'webpage_data.json')
            processed_data = crawler.process_wikipedia_data(raw_data)
            storage.save_to_json(processed_data, 'processed_webpage_data.json')

In [13]:
# Κύρια συνάρτηση εκτέλεσης του προγράμματος
# Αρχικοποίηση εργαλείων εξόρυξης και αποθήκευσης δεδομένων
# Λίστα θεμάτων προς συλλογή από Wikipedia
# Επανάληψη συλλογής και επεξεργασίας δεδομένων για κάθε θέμα
# Αποθήκευση ακατέργαστων και επεξεργασμένων δεδομένων
# Δημιουργία ανεστραμμένου ευρετηρίου
# Φόρτωση και χτίσιμο του ευρετηρίου από τα επεξεργασμένα δεδομένα

In [14]:
if __name__ == "__main__":
    main()

Συλλογή Δεδομλενων για :Python_programming_language
Συλλογή Δεδομλενων για :Artificial_intelligence
Συλλογή Δεδομλενων για :Machine_learning
Συλλογή Δεδομλενων για :Data_science
Συλλογή Δεδομλενων για :Computer_programming
Συλλογή Δεδομλενων για :Java_programming_language
Συλλογή Δεδομλενων για :Database_management_system
Συλλογή Δεδομλενων για :Cloud_computing
Συλλογή Δεδομλενων για :Cybersecurity
Συλλογή Δεδομλενων για :Web_development
Συλλογή Δεδομλενων για :Computer_networks
Συλλογή Δεδομλενων για :Software_engineering
Συλλογή Δεδομλενων για :Internet_of_things
Συλλογή Δεδομλενων για :Big_data
Συλλογή Δεδομλενων για :Computer_graphics
Συλλογή Δεδομλενων για :Operating_system
Συλλογή Δεδομλενων για :Mobile_computing
Συλλογή Δεδομλενων για :Computer_architecture
Συλλογή Δεδομλενων για :Information_security
Συλλογή Δεδομλενων για :Distributed_computing
Συλλογή Δεδομλενων για :Algorithm
Συλλογή Δεδομλενων για :Data_mining
Συλλογή Δεδομλενων για :Blockchain
Συλλογή Δεδομλενων για :Deep_

In [15]:
import math
from typing import List, Dict, Any
from collections import defaultdict

In [16]:
class RankingEngine:
    def __init__(self, inverted_index):
        self.inverted_index = inverted_index
        self.doc_lengths = self._calculate_doc_lengths()
        
        # Λεξικό με τους διαθέσιμους αλγόριθμους κατάταξης
        self.ranking_algorithms = {
            'boolean': self.boolean_search,
            'tfidf': self.tfidf_search,
            'bm25': self.bm25_search
        }
    
    def _calculate_doc_lengths(self) -> Dict[int, int]:
        """Υπολογίζει το μήκος (αριθμό λέξεων) κάθε εγγράφου"""
        doc_lengths = defaultdict(int)
        for word in self.inverted_index.index:
            for doc_id, info in self.inverted_index.index[word]['documents'].items():
                doc_lengths[doc_id] += info['count']
        return doc_lengths

    def calculate_tf(self, term: str, doc_id: int) -> float:
        """Υπολογίζει το term frequency ενός όρου σε ένα έγγραφο"""
        if term not in self.inverted_index.index:
            return 0.0
        
        doc_info = self.inverted_index.index[term]['documents'].get(doc_id, {})
        if not doc_info:
            return 0.0
        
        return doc_info['count'] / self.doc_lengths[doc_id]

    def calculate_idf(self, term: str) -> float:
        """Υπολογίζει το inverse document frequency ενός όρου"""
        if term not in self.inverted_index.index:
            return 0.0
        
        total_docs = len(self.inverted_index.documents)
        docs_with_term = len(self.inverted_index.index[term]['documents'])
        
        return math.log(total_docs / (1 + docs_with_term))

    def tfidf_search(self, query_terms: List[str]) -> Dict[int, float]:
        """Υπολογίζει το TF-IDF score για κάθε έγγραφο"""
        scores = defaultdict(float)
        
        for term in query_terms:
            idf = self.calculate_idf(term)
            for doc_id in self.inverted_index.index.get(term, {}).get('documents', {}):
                tf = self.calculate_tf(term, doc_id)
                scores[doc_id] += tf * idf
        
        return dict(scores)

    def boolean_search(self, query_terms: List[str]) -> Dict[int, float]:
        """Boolean αναζήτηση (για συμβατότητα με το υπάρχον σύστημα)"""
        # Χρησιμοποιεί την υπάρχουσα boolean αναζήτηση και επιστρέφει 1.0 για κάθε έγγραφο
        results = {}
        for term in query_terms:
            docs = self.inverted_index.get_format_for_term(term).get('documents', {})
            for doc_id in docs:
                results[doc_id] = 1.0
        return results

    def bm25_search(self, query_terms: List[str]) -> Dict[int, float]:
        """
        Υλοποίηση του BM25 αλγορίθμου.
        BM25 παράμετροι: k1 = 1.5, b = 0.75 (τυπικές τιμές)
        """
        k1 = 1.5  # term frequency saturation parameter
        b = 0.75  # length normalization factor

        # Υπολογισμός του μέσου μήκους εγγράφων
        avg_doc_length = sum(self.doc_lengths.values()) / \
        len(self.doc_lengths) if self.doc_lengths else 0

        scores = defaultdict(float)
        N = len(self.inverted_index.documents)  # συνολικός αριθμός εγγράφων

        for term in query_terms:
            if term not in self.inverted_index.index:
                continue

            # Αριθμός εγγράφων που περιέχουν τον όρο
            n = len(self.inverted_index.index[term]['documents'])

            # Υπολογισμός IDF για BM25
            idf = math.log((N - n + 0.5) / (n + 0.5) + 1)

            for doc_id, doc_info in self.inverted_index.index[term]['documents'].items():
                # Term frequency στο τρέχον έγγραφο
                tf = doc_info['count']

                # Μήκος του τρέχοντος εγγράφου
                doc_length = self.doc_lengths[doc_id]

                # Υπολογισμός του BM25 score για τον όρο
                numerator = tf * (k1 + 1)
                denominator = tf + k1 * (1 - b + b * doc_length / avg_doc_length)

                score = idf * numerator / denominator
                scores[doc_id] += score

        return dict(scores)


    def rank_documents(self, query_terms: List[str], algorithm:str) -> Dict[int, Dict[str, Any]]:
        """
        Κατατάσσει τα έγγραφα με βάση τον επιλεγμένο αλγόριθμο
        """
        if algorithm not in self.ranking_algorithms:
            raise ValueError(f"Μη έγκυρος αλγόριθμος: {algorithm}")
            
        # Παίρνουμε τα scores από τον επιλεγμένο αλγόριθμο
        scores = self.ranking_algorithms[algorithm](query_terms)
        
        # Δημιουργούμε τα αποτελέσματα με τους τίτλους και τα scores
        results = {}
        for doc_id, score in sorted(scores.items(), key=lambda x: x[1], reverse=True):
            results[doc_id] = {
                'title': self.inverted_index.documents[doc_id]['title'],
                'score': score
            }
            
        return results

In [17]:
# Αρχικοποίηση του RankingEngine με το ανεστραμμένο ευρετήριο
# Υπολογισμός του μήκους κάθε εγγράφου με βάση τις λέξεις του

# Λεξικό με τους διαθέσιμους αλγόριθμους κατάταξης (boolean, tfidf, bm25)

# Υπολογισμός του μήκους (αριθμός λέξεων) κάθε εγγράφου
# Διατρέχουμε το ευρετήριο για να υπολογίσουμε το πλήθος των λέξεων σε κάθε έγγραφο

# Υπολογισμός του TF (Term Frequency) για έναν όρο σε ένα έγγραφο
# Επιστρέφουμε την αναλογία των εμφανίσεων του όρου στο έγγραφο προς το συνολικό αριθμό λέξεων

# Υπολογισμός του IDF (Inverse Document Frequency) για έναν όρο
# Χρησιμοποιούμε τον τύπο για το IDF με βάση τον αριθμό των εγγράφων που περιέχουν τον όρο

# Υλοποίηση του αλγορίθμου TF-IDF για την κατάταξη των εγγράφων
# Υπολογίζουμε το TF-IDF score για κάθε έγγραφο και επιστρέφουμε τα αποτελέσματα

# Boolean αναζήτηση: επιστρέφει 1.0 για κάθε έγγραφο που περιέχει τον όρο
# Χρησιμοποιείται για συμβατότητα με το υπάρχον σύστημα αναζήτησης

# Υλοποίηση του αλγορίθμου BM25 για την κατάταξη των εγγράφων
# Υπολογισμός του BM25 score για κάθε έγγραφο με τις παραμέτρους k1 και b

# Υπολογισμός του BM25 score για έναν όρο σε κάθε έγγραφο με βάση την συχνότητα και το μήκος του εγγράφου

# Κατάταξη των εγγράφων με βάση τον επιλεγμένο αλγόριθμο
# Επιστρέφουμε τα αποτελέσματα ταξινομημένα κατά βαθμολογία με τον τίτλο και το score

In [18]:
class SearchEngine:
    def __init__(self, inverted_index:InvertedIndex):
        self.inverted_index = inverted_index
        self.lemmatizer = WordNetLemmatizer()
        self.stop_words = set(stopwords.words('english'))
        self.operators = {'AND','OR','NOT','(',')'}
        
    def preprocess_query(self, query: str) -> List[str]:
        query = query.replace('(', ' ( ').replace(')', ' ) ')
        tokens = query.split()
        
        processed_tokens = []
        
        
        for i,token in enumerate(tokens):
            if token.upper() in self.operators:
                processed_tokens.append(token.upper())
            else:
                token = re.sub(r'[^a-zA-Z0-9\s]', '', token.lower())
                if len(token)>1 and token not in self.stop_words:
                    processed_tokens.append(self.lemmatizer.lemmatize(token,'v'))
                    if i + 1 < len(tokens) and tokens[i + 1].upper() not in self.operators:
                        processed_tokens.append('AND')

         
        return processed_tokens
    
    def search(self, query: str) -> Dict[int, Dict[str, Any]]:
        tokens = self.preprocess_query(query)
    
        if not tokens:
            return {}

        # Αν έχουμε μόνο μία λέξη
        if len(tokens) == 1:
            result = self.inverted_index.get_format_for_term(tokens[0])
            return result.get('documents', {})

    
        current_docs = None
        current_op = None
        i = 0

        while i < len(tokens):
            token = tokens[i]
        
            if token in {'AND', 'OR', 'NOT'}:
                current_op = token
                i += 1
                continue

            term_docs = set(self.inverted_index.get_format_for_term(token).get('documents', {}).keys())

            if current_docs is None:
                current_docs = term_docs
            else:
                if current_op == 'AND':
                    current_docs = current_docs & term_docs
                elif current_op == 'OR':
                    current_docs = current_docs | term_docs
                elif current_op == 'NOT':
                    current_docs = current_docs - term_docs

            i += 1

        results = {}
        if current_docs:
            for doc_id in current_docs:
                results[doc_id] = {
                    'title': self.inverted_index.documents[doc_id]['title'],
                    'count': 1
                    }   

        return results
            
        
    
    def run(self):  
        while True:
            query = input("\nΕρώτημα: ").strip()
            if query.lower() == 'exit':
                break
            
            results = self.search(query)
            if results:
                print(f"\nΒρέθηκαν {len(results)} αποτελέσματα:")
                for doc_id, doc_info in results.items():
                    print(f"- {doc_info['title']}")
            else:
                print("Δεν βρέθηκαν αποτελέσματα.")

In [19]:
# Αρχικοποίηση του SearchEngine με το ανεστραμμένο ευρετήριο
# Δημιουργία λέξη προς λέξη των λεξιλογικών τύπων και φόρτωμα stopwords

# Προεπεξεργασία του ερωτήματος: αναγνώριση και επεξεργασία των τελεστών (AND, OR, NOT)
# Αντικατάσταση ειδικών χαρακτήρων και μετατροπή των λέξεων σε μορφή λεξιλογικής ρίζας
# Προσθήκη του τελεστή AND μετά από κάθε λέξη εκτός αν ακολουθείτε από τελεστή

# Εκτέλεση αναζήτησης βάσει του επεξεργασμένου ερωτήματος
# Αν το ερώτημα περιέχει μόνο μία λέξη, επιστρέφουμε τα έγγραφα που περιέχουν τον όρο

# Ανάλυση του ερωτήματος με τη χρήση τελεστών (AND, OR, NOT)
# Εφαρμογή του λογικού τελεστή για την επιλογή των εγγράφων που ικανοποιούν το ερώτημα

# Δημιουργία αποτελεσμάτων και επιστροφή των εγγράφων που πληρούν τα κριτήρια
# Επιστρέφουμε τα αποτελέσματα με τίτλους και πλήθος για κάθε έγγραφο

# Εκτέλεση του search engine σε συνεχή βάση μέσω της μεθόδου run
# Λήψη και εκτέλεση του ερωτήματος από τον χρήστη, με δυνατότητα εξόδου αν πληκτρολογηθεί 'exit'

In [20]:
def main():
    # Αρχικοποίηση του inverted index
    inverted_index = InvertedIndex()
    inverted_index.load_processed_data('processed_webpage_data.json')
    inverted_index.build_index()
    
    # Αρχικοποίηση της μηχανής αναζήτησης
    search_engine = SearchEngine(inverted_index)
    
    # Αρχικοποίηση της μηχανής κατάταξης
    ranking_engine = RankingEngine(inverted_index)
    
    print("\nΔιαθέσιμοι αλγόριθμοι αναζήτησης:")
    print("1. Boolean Search")
    print("2. TF-IDF Ranking")
    print("3. BM25 Ranking")
    
    while True:
        try:
            choice = input("\nΕπιλέξτε αλγόριθμο (1-3) ή 'exit' για έξοδο: ")
            
            if choice.lower() == 'exit':
                break
                
            if choice == '1':
                search_engine.run()  # Χρήση του υπάρχοντος boolean search
            elif choice == '2':
                algorithm = 'tfidf'
                query = input("\nΕρώτημα: ").strip()
                
                if query.lower() == 'exit':
                    break
                    
                # Προεπεξεργασία του ερωτήματος
                query_terms = search_engine.preprocess_query(query)
                results = ranking_engine.rank_documents(query_terms, algorithm)
                
                if results:
                    print(f"\nΒρέθηκαν {len(results)} αποτελέσματα:")
                    for doc_id, info in results.items():
                        print(f"- {info['title']} (score: {info['score']:.4f})")
                else:
                    print("Δεν βρέθηκαν αποτελέσματα.")
            elif choice =='3':
                algorithm = 'bm25'
                query = input("\nΕρώτημα: ").strip()
                
                if query.lower() == 'exit':
                    break
                    
                # Προεπεξεργασία του ερωτήματος
                query_terms = search_engine.preprocess_query(query)
                results = ranking_engine.rank_documents(query_terms, algorithm)
                
                if results:
                    print(f"\nΒρέθηκαν {len(results)} αποτελέσματα:")
                    for doc_id, info in results.items():
                        print(f"- {info['title']} (score: {info['score']:.4f})")
                else:
                    print("Δεν βρέθηκαν αποτελέσματα.")
            else:
                continue
        except KeyboardInterrupt:
            print("\nΈξοδος από το πρόγραμμα...")
            break
        except Exception as e:
            print(f"Σφάλμα: {e}")

In [21]:
# Αρχικοποίηση του inverted index
# Φόρτωση επεξεργασμένων δεδομένων από αρχείο και δημιουργία του ευρετηρίου

# Αρχικοποίηση της μηχανής αναζήτησης
# Αρχικοποίηση της μηχανής κατάταξης

# Εκτύπωση διαθέσιμων αλγορίθμων αναζήτησης (Boolean, TF-IDF, BM25)

# Εκκίνηση του βρόχου για την επιλογή αλγορίθμου από τον χρήστη
# Αν ο χρήστης επιλέξει 'exit', το πρόγραμμα τερματίζεται

# Επιλογή Boolean Search (1)
# Εκτέλεση του boolean search μέσω της μεθόδου run από τη μηχανή αναζήτησης

# Επιλογή TF-IDF Ranking (2)
# Προεπεξεργασία του ερωτήματος και κατάταξη των αποτελεσμάτων χρησιμοποιώντας τον αλγόριθμο TF-IDF

# Επιλογή BM25 Ranking (3)
# Προεπεξεργασία του ερωτήματος και κατάταξη των αποτελεσμάτων χρησιμοποιώντας τον αλγόριθμο BM25

# Ανάλογα με τον αλγόριθμο, επιστρέφουμε τα αποτελέσματα με τίτλους και σκορ
# Αν δεν βρεθούν αποτελέσματα, ενημερώνουμε τον χρήστη

# Αν παρουσιαστεί KeyboardInterrupt, τερματίζουμε το πρόγραμμα
# Αν υπάρξει κάποιο άλλο σφάλμα, το εκτυπώνουμε

In [22]:
if __name__ == "__main__":
    main()


Διαθέσιμοι αλγόριθμοι αναζήτησης:
1. Boolean Search
2. TF-IDF Ranking
3. BM25 Ranking



Επιλέξτε αλγόριθμο (1-3) ή 'exit' για έξοδο:  1

Ερώτημα:  coin



Βρέθηκαν 11 αποτελέσματα:
- Artificial intelligence - Wikipedia
- Machine learning - Wikipedia
- Computer programming - Wikipedia
- Database - Wikipedia
- Computer security - Wikipedia
- Internet of things - Wikipedia
- Big data - Wikipedia
- Computer graphics - Wikipedia
- Information security - Wikipedia
- Data mining - Wikipedia
- Blockchain - Wikipedia



Ερώτημα:  python AND big data



Βρέθηκαν 1 αποτελέσματα:
- Artificial intelligence - Wikipedia


...


Επιλέξτε αλγόριθμο (1-3) ή 'exit' για έξοδο:  2

Ερώτημα:  java



Βρέθηκαν 6 αποτελέσματα:
- Java (programming language) - Wikipedia (score: 0.0860)
- Python (programming language) - Wikipedia (score: 0.0013)
- Computer architecture - Wikipedia (score: 0.0011)
- Operating system - Wikipedia (score: 0.0008)
- Data mining - Wikipedia (score: 0.0008)
- Computer programming - Wikipedia (score: 0.0006)



Επιλέξτε αλγόριθμο (1-3) ή 'exit' για έξοδο:  3

Ερώτημα:  coin



Βρέθηκαν 11 αποτελέσματα:
- Internet of things - Wikipedia (score: 1.0275)
- Data mining - Wikipedia (score: 1.0078)
- Computer programming - Wikipedia (score: 0.9435)
- Information security - Wikipedia (score: 0.7546)
- Blockchain - Wikipedia (score: 0.7363)
- Database - Wikipedia (score: 0.7127)
- Big data - Wikipedia (score: 0.6942)
- Computer graphics - Wikipedia (score: 0.6861)
- Machine learning - Wikipedia (score: 0.6218)
- Computer security - Wikipedia (score: 0.5327)
- Artificial intelligence - Wikipedia (score: 0.4836)



Επιλέξτε αλγόριθμο (1-3) ή 'exit' για έξοδο:  exit


In [23]:
import numpy as np
from typing import List, Dict, Set
from collections import defaultdict

In [24]:
class SearchEvaluator:
    def __init__(self, search_engine, ranking_engine):
        self.search_engine = search_engine
        self.ranking_engine = ranking_engine
        
        # Δημιουργία test queries με τα αναμενόμενα σχετικά έγγραφα
        self.test_queries = {
            "python programming": {
                "relevant_docs": {
                    "Python (programming language) - Wikipedia",
                    "Computer programming - Wikipedia"
                },
                "description": "Basic programming query"
            },
            "machine learning AI": {
                "relevant_docs": {
                    "Machine learning - Wikipedia",
                    "Artificial intelligence - Wikipedia",
                    "Deep learning - Wikipedia"
                },
                "description": "AI-related query"
            },
            "database security": {
                "relevant_docs": {
                    "Database management system - Wikipedia",
                    "Computer security - Wikipedia",
                    "Information security - Wikipedia"
                },
                "description": "Security-focused query"
            },
            "web development": {
                "relevant_docs": {
                    "Web development - Wikipedia",
                    "Software engineering - Wikipedia",
                    "Computer programming - Wikipedia"
                },
                "description": "Web development query"
            },
            "cloud computing networks": {
                "relevant_docs": {
                    "Cloud computing - Wikipedia",
                    "Computer networks - Wikipedia",
                    "Distributed computing - Wikipedia"
                },
                "description": "Network-related query"
            }
        }

    def precision(self, retrieved: Set[str], relevant: Set[str]) -> float:
        """Υπολογίζει την ακρίβεια (precision)"""
        if not retrieved:
            return 0.0
        return len(retrieved.intersection(relevant)) / len(retrieved)

    def recall(self, retrieved: Set[str], relevant: Set[str]) -> float:
        """Υπολογίζει την ανάκληση (recall)"""
        if not relevant:
            return 0.0
        return len(retrieved.intersection(relevant)) / len(relevant)

    def f1_score(self, precision: float, recall: float) -> float:
        """Υπολογίζει το F1-score"""
        if precision + recall == 0:
            return 0.0
        return 2 * (precision * recall) / (precision + recall)

    def average_precision(self, retrieved: List[str], relevant: Set[str]) -> float:
        """Υπολογίζει το average precision για ένα query"""
        hits = 0
        sum_precision = 0.0
        
        for i, doc in enumerate(retrieved, 1):
            if doc in relevant:
                hits += 1
                sum_precision += hits / i
                
        return sum_precision / len(relevant) if relevant else 0.0

    def evaluate_algorithm(self, algorithm: str = 'tfidf', k: int = 10) -> Dict:
        """Αξιολογεί έναν συγκεκριμένο αλγόριθμο κατάταξης"""
        metrics = defaultdict(list)
        
        print(f"\nΛεπτομερή αποτελέσματα για {algorithm}:")
        print("-" * 40)
        
        for query, info in self.test_queries.items():
            print(f"\nQuery: {query}")
            
            # Λήψη αποτελεσμάτων
            if algorithm == 'boolean':
                results = self.search_engine.search(query)
            else:
                query_terms = self.search_engine.preprocess_query(query)
                results = self.ranking_engine.rank_documents(query_terms, algorithm)
            
            # Μετατροπή αποτελεσμάτων σε τίτλους
            retrieved_titles = {self.search_engine.inverted_index.documents[doc_id]['title'] 
                              for doc_id in list(results.keys())[:k]}
            
            print(f"Retrieved titles: {retrieved_titles}")
            print(f"Relevant docs: {info['relevant_docs']}")
            
            # Υπολογισμός μετρικών
            prec = self.precision(retrieved_titles, info['relevant_docs'])
            rec = self.recall(retrieved_titles, info['relevant_docs'])
            f1 = self.f1_score(prec, rec)
            ap = self.average_precision(list(retrieved_titles), info['relevant_docs'])
            
            print(f"Precision: {prec:.4f}")
            print(f"Recall: {rec:.4f}")
            print(f"F1: {f1:.4f}")
            print(f"AP: {ap:.4f}")
            
            metrics['precision'].append(prec)
            metrics['recall'].append(rec)
            metrics['f1'].append(f1)
            metrics['ap'].append(ap)
        
        # Υπολογισμός μέσων τιμών
        return {
            'mean_precision': np.mean(metrics['precision']),
            'mean_recall': np.mean(metrics['recall']),
            'mean_f1': np.mean(metrics['f1']),
            'map': np.mean(metrics['ap']),
            'details': metrics
        }
        """Αξιολογεί έναν συγκεκριμένο αλγόριθμο κατάταξης"""
        metrics = defaultdict(list)
        
        for query, info in self.test_queries.items():
            # Λήψη αποτελεσμάτων
            if algorithm == 'boolean':
                results = self.search_engine.search(query)
            else:
                query_terms = self.search_engine.preprocess_query(query)
                results = self.ranking_engine.rank_documents(query_terms, algorithm)
            
            # Μετατροπή αποτελεσμάτων σε τίτλους
            retrieved_titles = {self.search_engine.inverted_index.documents[doc_id]['title'] 
                              for doc_id in list(results.keys())[:k]}
            
            # Υπολογισμός μετρικών
            prec = self.precision(retrieved_titles, info['relevant_docs'])
            rec = self.recall(retrieved_titles, info['relevant_docs'])
            f1 = self.f1_score(prec, rec)
            ap = self.average_precision(list(retrieved_titles), info['relevant_docs'])
            
            metrics['precision'].append(prec)
            metrics['recall'].append(rec)
            metrics['f1'].append(f1)
            metrics['ap'].append(ap)
            
        # Υπολογισμός μέσων τιμών
        return {
            'mean_precision': np.mean(metrics['precision']),
            'mean_recall': np.mean(metrics['recall']),
            'mean_f1': np.mean(metrics['f1']),
            'map': np.mean(metrics['ap']),  # Mean Average Precision
            'details': metrics
        }

    def compare_algorithms(self, k: int = 10):
        """Συγκρίνει όλους τους διαθέσιμους αλγόριθμους"""
        algorithms = ['boolean', 'tfidf', 'bm25']
        results = {}
        
        for alg in algorithms:
            results[alg] = self.evaluate_algorithm(alg, k)
            
        self.print_evaluation_results(results)
        
    def print_evaluation_results(self, results: Dict):
        """Εκτύπωση των αποτελεσμάτων αξιολόγησης"""
        print("\nΑξιολόγηση Αλγορίθμων Αναζήτησης")
        print("=" * 50)
        
        for alg, metrics in results.items():
            print(f"\nΑλγόριθμος: {alg.upper()}")
            print("-" * 30)
            print(f"Mean Precision: {metrics['mean_precision']:.4f}")
            print(f"Mean Recall: {metrics['mean_recall']:.4f}")
            print(f"Mean F1-Score: {metrics['mean_f1']:.4f}")
            print(f"MAP: {metrics['map']:.4f}")

In [25]:
# Αρχικοποίηση της SearchEvaluator με τη μηχανή αναζήτησης και την μηχανή κατάταξης

# Δημιουργία λεξικού με τα queries για αξιολόγηση και τα αναμενόμενα σχετικά έγγραφα

# Υπολογισμός του precision για τα ανακτηθέντα έγγραφα σε σχέση με τα σχετικά έγγραφα
# Υπολογισμός του recall για τα ανακτηθέντα έγγραφα σε σχέση με τα σχετικά έγγραφα

# Υπολογισμός του F1-score ως το αρμονικό μέσο του precision και recall

# Υπολογισμός του average precision για το κάθε query, μετρώντας την ακρίβεια για κάθε ανακτηθέν έγγραφο

# Αξιολόγηση ενός συγκεκριμένου αλγορίθμου κατάταξης (boolean, tfidf, bm25)
# Υπολογισμός και εκτύπωση μετρικών όπως precision, recall, F1 και average precision

# Υπολογισμός των μέσων τιμών για όλες τις μετρικές που υπολογίστηκαν κατά την αξιολόγηση

# Σύγκριση όλων των διαθέσιμων αλγορίθμων αναζήτησης και εκτύπωση των αποτελεσμάτων

# Εκτύπωση των αποτελεσμάτων αξιολόγησης με τις μέσες τιμές των μετρικών για κάθε αλγόριθμο

In [26]:
def main():
    # Αρχικοποίηση των απαραίτητων components
    inverted_index = InvertedIndex()
    inverted_index.load_processed_data('processed_webpage_data.json')
    inverted_index.build_index()
    
    search_engine = SearchEngine(inverted_index)
    ranking_engine = RankingEngine(inverted_index)
    
    # Δημιουργία του evaluator
    evaluator = SearchEvaluator(search_engine, ranking_engine)
    
    # Εκτέλεση της αξιολόγησης
    print("Εκτέλεση αξιολόγησης συστήματος...")
    evaluator.compare_algorithms(k=10)

In [27]:
if __name__ == "__main__":
    main()

Εκτέλεση αξιολόγησης συστήματος...

Λεπτομερή αποτελέσματα για boolean:
----------------------------------------

Query: python programming
Retrieved titles: {'Python (programming language) - Wikipedia', 'Artificial intelligence - Wikipedia'}
Relevant docs: {'Python (programming language) - Wikipedia', 'Computer programming - Wikipedia'}
Precision: 0.5000
Recall: 0.5000
F1: 0.5000
AP: 0.5000

Query: machine learning AI
Retrieved titles: {'Python (programming language) - Wikipedia', 'Machine learning - Wikipedia', 'Deep learning - Wikipedia', 'Internet of things - Wikipedia', 'Artificial intelligence - Wikipedia', 'Data mining - Wikipedia'}
Relevant docs: {'Machine learning - Wikipedia', 'Artificial intelligence - Wikipedia', 'Deep learning - Wikipedia'}
Precision: 0.5000
Recall: 1.0000
F1: 0.6667
AP: 0.5889

Query: database security
Retrieved titles: {'Python (programming language) - Wikipedia', 'Internet of things - Wikipedia', 'Artificial intelligence - Wikipedia', 'Cloud computing -